In [ ]:
import numpy as np
import xarray as xr

from eval_utilities import spatial_temporal_metrics as stm
from eval_utilities import visualization as vis
import matplotlib.pyplot as plt

In [ ]:
import yaml
with open(f"config.yaml") as stream:
    try:
        CONFIG = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [ ]:
ds_ref = xr.open_zarr(CONFIG["path_ec_euro"]).sel(time=slice("2021-01-01T00", "2022-11-30T00"))
ds_mod = xr.open_zarr(CONFIG["path_xgb_diag_v2"]).sel(time=slice("2021-01-01T00", "2022-11-30T00"))

# Soil Moisture

## Field Capacity vs. Average Soil Moisture

In [ ]:
# Climate info:
field_cap = ds_ref.clim_data.sel(clim_variable="clim_theta_cap")
wilt_point = ds_ref.clim_data.sel(clim_variable="clim_theta_pwp")

# Average over time and soil moisture levels:
avg_ref = ds_ref.sel(variable=["swvl1","swvl2","swvl3"]).data.mean(dim=("time", "variable"))
avg_mod = ds_mod.sel(variable=["swvl1","swvl2","swvl3"]).data.mean(dim=("time", "variable"))


# Assemble data for boxplots:
fcaps = sorted(set(field_cap.values))
boxes_ref = []
boxes_mod = []
for fcap in fcaps:
    _ = avg_ref.where(ds_ref.clim_data.sel(clim_variable="clim_theta_cap") == fcap).values
    _ = _[~np.isnan(_)]
    boxes_ref.append(_)

    _ = avg_mod.where(ds_ref.clim_data.sel(clim_variable="clim_theta_cap") == fcap).values
    _ = _[~np.isnan(_)]
    boxes_mod.append(_)

In [ ]:
bpr = plt.boxplot(boxes_ref, positions=np.array(range(len(fcaps)))*2.0-0.4, widths=0.6)
bpm = plt.boxplot(boxes_mod, positions=np.array(range(len(fcaps)))*2.0+0.4, widths=0.6)
plt.xticks(range(0, len(fcaps) * 2, 2), fcaps)

# Distinguish mod and ref:
plt.setp(bpr['medians'], color="tab:orange", label="ref")
plt.setp(bpm['medians'], color="tab:blue", label="mod")

# Get rid of duplicates in the legend:
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlabel("Field Capacity")
plt.ylabel("Average (time and vert) Soil Moisture")
plt.show()

## Sensitivity of Soil Moisture to Soil Properties

This first part is modified from "inference_xgboost.iypnb". It contains all necessary ingredients for running the inference.

In [ ]:
import xgboost as xgb
from tqdm import tqdm

from eval_utilities.EclandPointDataset import EcDataset

In [ ]:
data_path = "/data/ecland_i6aj_o400_2010_2022_6h_euro.zarr"
model_path = "/home/ch23/weights_ch23/euro_xgb_train_2019_val_2020_all_variables.json"
spatial_encoding = False
temporal_encoding = False

# Dataset:
ds_inf = EcDataset(start_year = 2020, end_year = 2022, root = data_path, roll_out = 1, 
                   #clim_features=CONFIG["clim_feats"], dynamic_features=CONFIG["dynamic_feats"],
                   #target_prog_features=CONFIG["targets_prog"], target_diag_features=CONFIG["targets_diag"],
                   is_add_lat_lon = spatial_encoding, 
                   is_norm = True, 
                   point_dropout = 0.0)

# XGB Model:
model = xgb.XGBRegressor(n_estimators=1000, tree_method="hist", device="cuda")
model.load_model(model_path)

Output should neither be stored nor is it necessary to predict the whole data set. Instead, we want to run the same grid point with different parameters.
Here, we are modifying only field capacitiy (for now). To get the whole range, field capacity is varied from $0$ to $0.75$ in steps of $0.05$.
The test sites are repeated and stacked to run the inference for all variations at once.

In [ ]:
# Choose test sites:
x_coords = [63699, 62986, 62267, 85271] #Bonn, Juelich, Reading, Bologna
ii = np.searchsorted(ds_ref["x"], x_coords) #corresponding indices

# Get index of field capacity in clim input:
i_cap = np.where(np.array(CONFIG["clim_feats"]) == "clim_theta_cap")[0][0]

# Define new field capacities and normalize:
new_caps = (np.arange(0, 0.8, 0.05) - ds_inf.clim_means[i_cap]) / ds_inf.clim_stdevs[i_cap]


# Initial state
_, x_state, _, _, x_clim, _ = ds_inf[0]
x_state, x_clim = x_state.squeeze()[ii], x_clim.squeeze()[ii]
preds = [EcDataset.inv_transform(x_state, ds_inf.y_prog_means, ds_inf.y_prog_stdevs)]


# Change initial state to contain duplicates of the test sites with the new field capacities:
new_states = []
new_clims = []
for i_site in range(len(x_coords)):
    new_states.append(np.vstack([x_state[i_site]] * len(new_caps)))
    new_clims.append(np.vstack([x_clim[i_site]] * len(new_caps)))
    new_clims[-1][:,i_cap] = new_caps

x_state = np.vstack(new_states)
x_clim = np.vstack(new_clims)

In [ ]:
# Constrained to be applied to each time step:
def apply_physical_constraints(x_state):
    x_state[:, np.array(ds_inf.target_prog_features)!="e"] = np.clip(x_state[:, np.array(ds_inf.target_prog_features)!="e"], 0, None) # All variables except "e" are positive
    x_state[:,-1] = np.clip(x_state[:,-1], None, 100) # Snow cover cannot be higher than 100
    return x_state

# Inference
for i in tqdm(range(len(ds_inf)), desc="Running ECLand emulator..."):
    x_met, _, _, _, _, _ = ds_inf[i]
    x_met = x_met.squeeze()[ii]
    
    # Duplicate weather conditions to match the structure above:
    x_met = np.vstack([np.vstack([x_met[i_site]] * len(new_caps)) for i_site in range(len(x_coords))])
    
    X = np.concatenate((x_met, x_state, x_clim), axis=1)
    y_pred = model.predict(X)
    y_state_inc_pred = y_pred[:,:len(ds_inf.target_prog_features)]
    y_state_inc_pred = EcDataset.inv_transform(y_state_inc_pred, ds_inf.y_prog_inc_mean, ds_inf.y_prog_inc_std) # Unnormalize so that it can be added to the normalized state vector
    x_state += y_state_inc_pred
    x_state = apply_physical_constraints(EcDataset.inv_transform(x_state, ds_inf.y_prog_means, ds_inf.y_prog_stdevs)) # Unnormalize updated state vector and apply consistency constraints
    preds.append(x_state)
    x_state = EcDataset.transform(x_state, ds_inf.y_prog_means, ds_inf.y_prog_stdevs) # Re-normalize state vector for next iteration

result = np.stack(preds[1:])

In [ ]:
fig, ax = plt.subplots()
i_swvl1 = np.where(np.array(ds_inf.target_prog_features) == "swvl1")[0][0]

for i in range(12):
    ax.plot(ds_inf["time"], result[:,i,i_swvl1])

ax.set(ylabel="swvl1")
plt.tick_params(rotation=45)

plt.show()

## Sensitivity of Evaporation to Soil Moisture

In [ ]:
x_coords = {"Bonn": 63699, "Juelich": 62986, "Reading": 62267, "Bologna": 85271}

mod_lhf = ds_mod.sel(variable="slhf").data / 3600.
ref_lhf = ds_ref.sel(variable="slhf").data / 3600.
R_n = ds_ref.sel(variable="met_lwdown").data + ds_ref.sel(variable="met_swdown").data

mod_sm = ds_mod.sel(variable=["swvl1","swvl2"]).data.mean(dim="variable")
ref_sm = ds_ref.sel(variable=["swvl1","swvl2"]).data.mean(dim="variable")

In [ ]:
for name,x in x_coords.items():
    fig, ax = plt.subplots()

    ax.scatter(mod_sm.sel(x=x), -mod_lhf.sel(x=x) / R_n.sel(x=x), marker=".")
    ax.scatter(ref_sm.sel(x=x), -ref_lhf.sel(x=x) / R_n.sel(x=x), marker=".")

    ax.axvline(wilt_point.sel(x=x), color="tab:grey")
    ax.text(wilt_point.sel(x=x), 0.99, 'WILT', color='tab:grey', ha='right', va='top', rotation=90, transform=ax.get_xaxis_transform())

    ax.set(title=f"SM Regimes for {name}", xlabel="Average Soil Moisture lvl 1-2", ylabel="Latent Flux / Incoming Radiation")
    plt.show()

# Water balance

In [ ]:
# test the water balance equation: delta S=P-E-R -- long-term delta S should be 0 -- yearly?
eval_ds = xr.open_zarr("/data/ch23/euro_xgb_train_2010_2019_val_2020_2020_diagnostic_v2.zarr")  # Global dataset
# train_ds = xr.open_zarr("/data/ecland_i6aj_o400_2010_2022_6h_euro.zarr")  # Europe subset
eval_ds

In [ ]:
# Plotting 
(eval_ds
 .data
 .isel(time=2)
 .sel(variable="e").to_dataset()
 .plot.scatter(x="lon", y="lat", hue="data", s=10, edgecolors="none",figsize=(12,6))
)
plt.show()

In [ ]:
train_ds = xr.open_zarr("/data/ecland_i6aj_o400_2010_2022_6h_euro.zarr")

In [ ]:
train_ds

In [ ]:
# Plotting 
(train_ds
 .data
 .isel(time=102)
 .sel(variable="e").to_dataset()
 .plot.scatter(x="lon", y="lat", hue="data", s=10, edgecolors="none",vmin=-0.003,vmax=0,figsize=(12,6))
)
plt.show()

In [ ]:
# Yearly Water Balance Evaluation...Test in 2020
# P-ET-R=delta S
# what is the unit? 
# P seems to be much smaller than E?
# Discharge is very large

start_date = '2020-01-01'
end_date = '2020-11-30 12:00:00'

train_ds_p = train_ds.data.sel(variable="met_rainf",time=slice(start_date, end_date))
train_ds_e= train_ds.data.sel(variable="e",time=slice(start_date, end_date))
train_ds_dis= train_ds.data.sel(variable="dis",time=slice(start_date, end_date))

eval_ds_e= eval_ds.data.sel(variable="e",time=slice(start_date, end_date))
eval_ds_dis= eval_ds.data.sel(variable="dis",time=slice(start_date, end_date))

In [ ]:
# calculate yearly sum/mean

train_ds_p.sum(dim='time').values*1000
train_ds_e.sum(dim='time').values*1000
train_ds_dis.mean(dim='time').values